<a href="https://colab.research.google.com/github/shruthimohan03/video-summarizer/blob/main/GMM_with_time_based_chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [8]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

# Download required NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Define phrases to remove (extended list)
REMOVE_PHRASES = [
    r'\b[Ii] repeat\b',
    r'\buh\b', r'\bum\b', r'\bhmm\b', r'\buhh\b', r'\bmmm\b',
    r'\bokay\b', r'\bOkay\b', r'\bOK\b', r'\bok\b',
    r'\bso\b', r'\bSo\b',
    r'\byou know\b', r'\blike\b', r'\bbasically\b',
    r'\bactually\b', r'\bkind of\b', r'\bsort of\b',
    r'\bat the end of the day\b', r'\bwell\b', r'\bright\b',
    r'\bjust\b', r'\bnow\b', r'\bthen\b',
    r'\bin particular\b', r'\bin general\b',
    r'\bI mean\b', r'\blet\'s see\b', r'\bas I said\b',
    r'\bremember\b', r'\bto be honest\b', r'\bI suppose\b',
    r'\bin fact\b', r'\bI think\b', r'\bfor example\b',
    r'\bin this particular\b', r'\bfor instance\b',
    r'\bas you know\b', r'\bas you can see\b',
    r'\blet me just\b', r'\bI\'ll just\b',
    r'\bif you think about it\b', r'\bas we discussed\b',
    r'\bsuppose\b', r'\bby that I mean\b',
    r'\bspecifically\b', r'\bbriefly\b',
    r'\bas you have seen\b', r'\bas we have discussed\b',
    r'\bas I have mentioned\b',
    r'\bnow coming back to\b', r'\bnow, coming back to\b',
    r'\btoday in this lecture\b', r'\bin this lecture\b',
    r'\bas the name would suggest\b', r'\bremember the name\b',
    r'\bthat is\b', r'\bthat means\b',
    r'\bas I was saying\b', r'\bas we were discussing\b'
]

REMOVE_REGEX = re.compile('|'.join(REMOVE_PHRASES))

def load_transcript(file_path):
    """Load and preprocess a transcript from a text file."""
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        return preprocess_transcript(text)

def preprocess_transcript(text):
    """Apply comprehensive preprocessing to transcript text."""
    # Convert to lowercase (optional, depending on requirements)
    text = text.lower()

    # Remove filler phrases
    text = REMOVE_REGEX.sub('', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Fix common transcription issues
    text = fix_transcription_errors(text)

    # Normalize punctuation
    text = normalize_punctuation(text)

    # Split into sentences (using NLTK for better segmentation)
    sentences = sent_tokenize(text)

    # Optional: Remove short sentences (likely fragments)
    sentences = [s for s in sentences if len(s.split()) > 3]

    return sentences

def fix_transcription_errors(text):
    """Fix common transcription errors."""
    # Fix repeated words (e.g., "the the")
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

    # Fix common ASR errors
    replacements = {
        'Sanda': 'Sandhi',
        'Sunday operation': 'Sandhi operation',
        'GIFs law': 'Zipf\'s law',
        'GIF\'s law': 'Zipf\'s law',
        'Heaps law': 'Heaps\' law',
        'Giph law': 'Zipf\'s law',
        'Tokenarage': 'Tokenizer',
        'tokenarize': 'tokenize',
        'lamtaization': 'lemmatization',
        'lamtization': 'lemmatization',
        'perqueous': 'previous',
        'griddle': 'greedy',
        'Eglidi': 'greedy',
        'on the relation': 'sandhi relation',
        'portus algorithm': 'Porter\'s algorithm',
        'clean it': 'kleene',
        'affix is': 'affixes',
        'prefossessing': 'preprocessing',
        'infix': 'infix',
        'perqueous': 'previous',
        'coppers': 'corpus'
    }

    for old, new in replacements.items():
        text = re.sub(r'\b' + re.escape(old) + r'\b', new, text)

    return text

def normalize_punctuation(text):
    """Standardize punctuation in transcript."""
    # Fix spacing around punctuation
    #text = re.sub(r'\s+([.,;:!?])', r'\1', text)

    # Standardize quotes
    text = re.sub(r'["""]', '"', text)
    text = re.sub(r"['\']", "'", text)

    # Fix ellipses
    text = re.sub(r'\.\.\.', 'â€¦', text)

    # Fix dashes
    text = re.sub(r'--', 'â€”', text)

    #remove commas
    text = re.sub(r',', '', text)
    return text

'''def remove_stopwords(sentences, exclude_list=None):
    """Remove stopwords from sentences."""
    stop_words = set(stopwords.words('english'))
    if exclude_list:
        stop_words = stop_words - set(exclude_list)

    result = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        filtered_words = [word for word in words if word.lower() not in stop_words]
        result.append(' '.join(filtered_words))

    return result

def stem_sentences(sentences):
    """Apply Porter stemming to sentences."""
    stemmer = PorterStemmer()
    result = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        stemmed_words = [stemmer.stem(word) for word in words]
        result.append(' '.join(stemmed_words))

    return result

def lemmatize_sentences(sentences):
    """Apply lemmatization to sentences."""
    lemmatizer = WordNetLemmatizer()
    result = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        result.append(' '.join(lemmatized_words))

    return result'''

# Example usage
if __name__ == "__main__":
    file_path = "/content/transcribed_text_nptel_video_1.txt"

    # Basic preprocessing
    sentences = load_transcript(file_path)
    print(f"Number of sentences after basic preprocessing: {len(sentences)}")

    # Optional: Further processing based on needs
    # sentences_without_stopwords = remove_stopwords(sentences)
    # lemmatized_sentences = lemmatize_sentences(sentences)
    # stemmed_sentences = stem_sentences(sentences)

    # Print first few sentences
    for i, sentence in enumerate(sentences[:15]):
        print(f"Sentence {i+1}: {sentence}")

Number of sentences after basic preprocessing: 320
Sentence 1: hello everyone welcome back to the final lecture of the first week.
Sentence 2: in the last lecture we were discussing about various empirical laws.
Sentence 3: gifs law and heaps law that how the words in the vocabulary are distributed in a corpus.
Sentence 4: and we saw that the distribution is not very uniform.
Sentence 5: there are certain words that are very very common.
Sentence 6: we saw that roughly 100 words in the vocabulary make for 50 percent of the corpus but by that i mean that number of tokens.
Sentence 7: and on the other hand there are 50 percent of the words in the vocabulary that occur only once.
Sentence 8: and we discussed what are the various relationships among the vocabulary size and the number of tokens that i observe in a corpus.
Sentence 9: and also how they grow with us to each other.
Sentence 10: and gifs law gives gave me a relation between the frequency and the rank of a word.
Sentence 11: we 

In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.mixture import GaussianMixture
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

def chunk_sentences(sentences, chunk_size=5):
    """Divide transcript sentences into overlapping chunks."""
    return [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size // 2)]

def get_gmm_clusters(embeddings, num_clusters):
    """Cluster embeddings using Gaussian Mixture Model."""
    gmm = GaussianMixture(n_components=num_clusters, random_state=42)
    gmm.fit(embeddings)
    labels = gmm.predict(embeddings)
    return labels

def extract_central_sentences(sentences, embeddings, labels):
    """Select the most central sentence from each cluster."""
    unique_clusters = np.unique(labels)
    summary = []

    for cluster in unique_clusters:
        indices = np.where(labels == cluster)[0]
        cluster_embeddings = embeddings[indices]
        centroid = np.mean(cluster_embeddings, axis=0)

        # Select the sentence closest to the centroid
        closest_idx = indices[np.argmin(np.linalg.norm(cluster_embeddings - centroid, axis=1))]
        summary.append(sentences[closest_idx])

    return summary

def extract_keywords(sentences):
    """Extract top keywords from each cluster using TF-IDF."""
    vectorizer = TfidfVectorizer(stop_words="english", max_features=5)  # Top 5 keywords
    tfidf_matrix = vectorizer.fit_transform(sentences)
    feature_names = vectorizer.get_feature_names_out()
    sorted_indices = np.argsort(-tfidf_matrix.toarray().sum(axis=0))[:5]
    return [feature_names[i] for i in sorted_indices]

def summarize_transcript(sentences, chunk_size, num_clusters):
    """Perform extractive summarization with clustering and print topics."""
    chunks = chunk_sentences(sentences, chunk_size)

    overall_summary = []
    clustered_sentences = defaultdict(list)

    for chunk in chunks:
        if len(chunk) < 3:  # Skip very small chunks
            overall_summary.extend(chunk)
            continue

        embeddings = np.array(model.encode(chunk))
        labels = get_gmm_clusters(embeddings, num_clusters)
        summary_sentences = extract_central_sentences(chunk, embeddings, labels)

        overall_summary.extend(summary_sentences)

        for i, label in enumerate(labels):
            clustered_sentences[label].append(chunk[i])

    # Print topics and their sentences
    for topic, topic_sentences in clustered_sentences.items():
        print(f"\nðŸŸ¢ Topic {topic}:")
        for s in topic_sentences:
            print(f"  - {s}")

        # Extract and print keywords
        keywords = extract_keywords(topic_sentences)
        print(f"  ðŸ”‘ Keywords: {', '.join(keywords)}")

    print("\nðŸ“Œ Final Summary:")
    print(" ".join(overall_summary))

    return " ".join(overall_summary)

summary = summarize_transcript(sentences, chunk_size=len(sentences)//3, num_clusters=3)



ðŸŸ¢ Topic 2:
  - hello everyone welcome back to the final lecture of the first week.
  - in the last lecture we were discussing about various empirical laws.
  - gifs law and heaps law that how the words in the vocabulary are distributed in a corpus.
  - and we saw that the distribution is not very uniform.
  - there are certain words that are very very common.
  - we saw that roughly 100 words in the vocabulary make for 50 percent of the corpus but by that i mean that number of tokens.
  - and on the other hand there are 50 percent of the words in the vocabulary that occur only once.
  - and we discussed what are the various relationships among the vocabulary size and the number of tokens that i observe in a corpus.
  - and also how they grow with us to each other.
  - and gifs law gives gave me a relation between the frequency and the rank of a word.
  - we will start with the basic key processing in language.
  - we will cover the basic concepts and what are the challenges that on

In [13]:
summary

'whenever i have a exclamation or question mark i can say probably this is the end of the sentence but is the case the same with a dot. there are two classes end of the sentence or not end of the sentence. in text processing if we face this problem in nearly every simple task that we are doing. again whenever i have the end of the sentence the next word starts with a capital. all these are my observations that i use as my features. my classifiers that i will build can be some rules that i write by hand some simple if-ten-else rules. my tokenization is to find out each of the 11 word tokens from the sentence. you will end up within hyphen. this problem you will see is again language dependent. this further complicates my problem of word segmentation or segmentation of. you might be familiar with the rules of sanskrit language. again when you are doing tokenization you have problem at how do i handle all these hyphens. this silly iteration can you think of some cases where the segmentati